# Saving and Loading Models
In this notebook, I'll show you how to save and load models with PyTorch. This is important because you'll often want to load previously trained models to use in making predictions or to continue training on new data.


In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

#import helper
from ipynb.fs.full import fc_model   #user specific model

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [6]:
#Here we can see one of the images.

image, label = next(iter(trainloader))
img_show(image[0,:]);

NameError: name 'img_show' is not defined

## Train a network
To make things more concise here, I moved the model architecture and training code from the last part to a file called fc_model. Importing this, we can easily create a fully-connected network with fc_model.Network, and train the network using fc_model.train. I'll use this model (once it's trained) to demonstrate how we can save and load models.


In [7]:
# Create the network, define the criterion and optimizer

model = fc_model.Network(784, 10, [512, 256, 128])  #input, output, list of hidden layers
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

fc_model.train(model, trainloader, testloader, criterion, optimizer, epochs=2)

Epoch: 1/2..  Training Loss: 1.740..  Test Loss: 1.009..  Test Accuracy: 0.624
Epoch: 1/2..  Training Loss: 1.035..  Test Loss: 0.742..  Test Accuracy: 0.723
Epoch: 1/2..  Training Loss: 0.850..  Test Loss: 0.666..  Test Accuracy: 0.734
Epoch: 1/2..  Training Loss: 0.781..  Test Loss: 0.629..  Test Accuracy: 0.758
Epoch: 1/2..  Training Loss: 0.752..  Test Loss: 0.618..  Test Accuracy: 0.771
Epoch: 1/2..  Training Loss: 0.714..  Test Loss: 0.616..  Test Accuracy: 0.762
Epoch: 1/2..  Training Loss: 0.706..  Test Loss: 0.587..  Test Accuracy: 0.785
Epoch: 1/2..  Training Loss: 0.721..  Test Loss: 0.564..  Test Accuracy: 0.793
Epoch: 1/2..  Training Loss: 0.686..  Test Loss: 0.589..  Test Accuracy: 0.782
Epoch: 1/2..  Training Loss: 0.691..  Test Loss: 0.539..  Test Accuracy: 0.805
Epoch: 1/2..  Training Loss: 0.609..  Test Loss: 0.515..  Test Accuracy: 0.807
Epoch: 1/2..  Training Loss: 0.608..  Test Loss: 0.523..  Test Accuracy: 0.812
Epoch: 1/2..  Training Loss: 0.606..  Test Loss: 0.5


## Saving and loading networks
As you can imagine, it's impractical to train a network every time you need to use it. Instead, we can save trained networks then load them later to train more or use them for predictions.

The parameters for PyTorch networks are stored in a model's state_dict. We can see the state dict contains the weight and bias matrices for each of our layers.


In [26]:
#print("Our model: \n\n", model, '\n')
#print("The state dict keys: \n\n", model.state_dict().keys())
print(model.state_dict().keys())

odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'output.weight', 'output.bias'])


The simplest thing to do is simply save the state dict with torch.save. For example, we can save it to a file 'checkpoint.pth'.


In [30]:
torch.save(model.state_dict(), 'checkpoint.pth')
print(model.state_dict().keys())

And to load the state dict in to the network, you do model.load_state_dict(state_dict).

In [33]:
model.load_state_dict(model.state_dict())

<All keys matched successfully>

Seems pretty straightforward, but as usual it's a bit more complicated. Loading the state dict works only if the model architecture is exactly the same as the checkpoint architecture. If I create a model with a different architecture, this fails.


In [37]:
# Try this
model = fc_model.Network(784, 10, [400, 200, 100])
# This will throw an error because the tensor sizes are wrong!
model.load_state_dict(model.state_dict())
print(model.state_dict().keys())

odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'output.weight', 'output.bias'])


This means we need to rebuild the model exactly as it was when trained. Information about the model architecture needs to be saved in the checkpoint, along with the state dict. To do this, you build a dictionary with all the information you need to compeletely rebuild the model.


In [40]:
checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

Now the checkpoint has all the necessary information to rebuild the trained model. You can easily make that a function if you want. Similarly, we can write a function to load checkpoints.


In [41]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [42]:
model = load_checkpoint('checkpoint.pth')
print(model)

Network(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=400, bias=True)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): Linear(in_features=200, out_features=100, bias=True)
  )
  (output): Linear(in_features=100, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [43]:
# Try this
model = fc_model.Network(784, 10, [512, 256, 128])
# This will throw an error because the tensor sizes are wrong!
model.load_state_dict(model.state_dict())
print(model.state_dict().keys())

odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'output.weight', 'output.bias'])


In [44]:
torch.save(model.state_dict(), 'checkpoint.pth')
print(model.state_dict().keys())

odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'output.weight', 'output.bias'])


In [46]:


print(model)

Network(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (output): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
